In [1]:
# Day 22 Part 1

import numpy as np
import re

class Change:
    def __init__(self, state, xlim, ylim, zlim):
        self.state, self.xlim, self.ylim, self.zlim = state, xlim, ylim, zlim
        
    def __str__ (self):
        return self.state + ' ' + str(self.xlim) + ' ' + str(self.ylim) + ' ' + str(self.zlim)

def read_input_data(filename):
    with open(filename, 'r') as in_file:
        raw_data = in_file.read().split('\n')
    
    p = re.compile("(on|off) x=(-?[0-9]*)..(-?[0-9]*),y=(-?[0-9]*)..(-?[0-9]*),z=(-?[0-9]*)..(-?[0-9]*)")
    data = []
    for line in raw_data:
        matches = p.findall(line)[0]
        data.append(Change(matches[0], (int(matches[1]), int(matches[2])), (int(matches[3]), int(matches[4])), (int(matches[5]), int(matches[6]))))
    
    return data

def count_on_cubes(data):
    reactor_state = np.zeros((101, 101, 101), dtype=int)
    for change in data:
        if change.xlim[0] > 50 or change.ylim[0] > 50 or change.zlim[0] > 50 or change.xlim[1] < -50 or change.ylim[1] < -50 or change.zlim[1] < -50:
            continue
        reactor_state[max(-50, change.xlim[0])+50:min(50, change.xlim[1])+51, max(-50, change.ylim[0])+50:min(50, change.ylim[1])+51, max(-50, change.zlim[0])+50:min(50, change.zlim[1])+51] = 1 if change.state == 'on' else 0

    return np.count_nonzero(reactor_state == 1)

# Test first
print("Running on test data. Verifying result...")
test_data = read_input_data('test_data/day22a.txt')
print(count_on_cubes(test_data) == 39)
test_data = read_input_data('test_data/day22b.txt')
print(count_on_cubes(test_data) == 590784)
print('')

# Now run on actual data
real_data = read_input_data('data/day22.txt')
result = count_on_cubes(real_data)
print("Puzzle answer is: " + str(result))

Running on test data. Verifying result...
True
True

Puzzle answer is: 615869


In [20]:
# Day 22 Part 2

class Section:
    def __init__(self, xlim, ylim, zlim):
        self.xlim, self.ylim, self.zlim = xlim, ylim, zlim
        
    def __str__ (self):
        return str(self.xlim) + ' ' + str(self.ylim) + ' ' + str(self.zlim)
    
    def get_size(self):
        return (self.xlim[1] - self.xlim[0] + 1) * (self.ylim[1] - self.ylim[0] + 1) * (self.zlim[1] - self.zlim[0] + 1)

    
def ranges_intersect(change, section):
    r = (len(range(max(change.xlim[0], section.xlim[0]), min(change.xlim[1], section.xlim[1]) + 1)) *
    len(range(max(change.ylim[0], section.ylim[0]), min(change.ylim[1], section.ylim[1]) + 1)) *
    len(range(max(change.zlim[0], section.zlim[0]), min(change.zlim[1], section.zlim[1]) + 1)) > 0)
    
    return r

def fully_contains(change, section):
    return change.xlim[0] < section.xlim[0] and change.xlim[1] > section.xlim[1] and change.ylim[0] < section.ylim[0] and change.ylim[1] > section.ylim[1] and change.zlim[0] < section.zlim[0] and change.zlim[1] > section.zlim[1]
    
def split(change, section):
    split_sections = []
    
    if change.zlim[0] > section.zlim[0] and change.zlim[0] <= section.zlim[1]:
        split_sections.append(Section(section.xlim, section.ylim, (section.zlim[0], change.zlim[0] - 1)))
        rest = Section(section.xlim, section.ylim, (change.zlim[0], section.zlim[1]))
        split_sections.extend(split(change, rest))
    elif change.zlim[1] >= section.zlim[0] and change.zlim[1] < section.zlim[1]:
        split_sections.append(Section(section.xlim, section.ylim, (change.zlim[1] + 1, section.zlim[1])))
        rest = Section(section.xlim, section.ylim, (section.zlim[0], change.zlim[1]))
        split_sections.extend(split(change, rest))
    elif change.ylim[0] > section.ylim[0] and change.ylim[0] <= section.ylim[1]:
        split_sections.append(Section(section.xlim, (section.ylim[0], change.ylim[0] - 1), section.zlim))
        rest = Section(section.xlim, (change.ylim[0], section.ylim[1]), section.zlim)
        split_sections.extend(split(change, rest))
    elif change.ylim[1] >= section.ylim[0] and change.ylim[1] < section.ylim[1]:
        split_sections.append(Section(section.xlim, (change.ylim[1] + 1, section.ylim[1]), section.zlim))
        rest = Section(section.xlim, (section.ylim[0], change.ylim[1]), section.zlim)
        split_sections.extend(split(change, rest))
    elif change.xlim[0] > section.xlim[0] and change.xlim[0] <= section.xlim[1]:
        split_sections.append(Section((section.xlim[0], change.xlim[0] - 1), section.ylim, section.zlim))
        rest = Section((change.xlim[0], section.xlim[1]), section.ylim, section.zlim)
        split_sections.extend(split(change, rest))
    elif change.xlim[1] >= section.xlim[0] and change.xlim[1] < section.xlim[1]:
        split_sections.append(Section((change.xlim[1] + 1, section.xlim[1]), section.ylim, section.zlim))
        rest = Section((section.xlim[0], change.xlim[1]), section.ylim, section.zlim)
        split_sections.extend(split(change, rest))
        
    return split_sections
   

def count_on_cubes_full(data):
    on_sections = []
    for change in data:
        new_sections = []
        for section in on_sections:
            if fully_contains(change, section):
                continue
            elif ranges_intersect(change, section):
                split_sections = split(change, section)
                new_sections.extend(split_sections)
            else:
                new_sections.append(section)
        if change.state == 'on':
            new_sections.append(Section(change.xlim, change.ylim, change.zlim))
        on_sections = new_sections
    
    # and then here we count the ones based on every section we found
    n_on_cubes = 0
    for section in on_sections:
        n_on_cubes += section.get_size()
    
    return n_on_cubes

# Test first
print("Running on test data. Verifying result...")
test_data = read_input_data('test_data/day22a.txt')
print(count_on_cubes_full(test_data) == 39)
test_data = read_input_data('test_data/day22c.txt')
print(count_on_cubes_full(test_data) == 2758514936282235)
print('')

# Now run on actual data
real_data = read_input_data('data/day22.txt')
result = count_on_cubes_full(real_data)
print("Puzzle answer is: " + str(result))

Running on test data. Verifying result...
39
True
2758514936282235
True

1323862415207825
Puzzle answer is: 1323862415207825
